# Explores osm_build_network

This notebook is for snapping of substations to lines

### Jupyter notebook needs to be improved (content and structure?). 

It seems also geoplot need to be added to the environment. How to: 
- One need to deinstall the whole environment locally and install everything from scratch from the environment.yaml. Thereby the environment.yaml needs now `geoplot`.
- Subsequently, one need to test if there are any conflicts using geoplot in the other scripts (geoplot might be in conflict with other packages)

In [12]:
import os
import matplotlib.pyplot as plt
import geopandas as gpd
#import geoplot
import pandas as pd
import numpy as np
import pandas as pd
import hvplot.pandas

import sys

# module_path = os.path.abspath(os.path.join('../../')) # To import helpers
# if module_path not in sys.path:
#     sys.path.append(module_path+"/pypsa-earth/scripts")
    
#from _helpers import sets_path_to_root

#sets_path_to_root("pypsa-earth-project")

# Africa shape data

In [4]:
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
# africa = world.query('continent == "Africa"')
boliviaplot = world.query('name == "Bolivia"')
# world

C:\Users\xenia\AppData\Local\Temp\ipykernel_3700\2054998892.py:1: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))


# Power data

In [20]:
# choose the option to get your plot
option = 'raw'
# option = 'clean'
option = "line2network"
raw_path = "resources/osm/raw"
clean_path = "resources/osm/clean"
b_network_path = "resources/base_network"
home = "c:/Users/xenia/Desktop/Thesis_Code/Code/pypsa-earth_working/pypsa-earth/"


if option == "raw":
    pathg = home + raw_path +"/all_raw_generators.geojson"
    paths = home + raw_path + + "/all_raw_substations.geojson"
    pathl = home + raw_path + + "/all_raw_lines.geojson"

    generators = gpd.read_file(pathg).set_crs(epsg=4326, inplace=True)
    substations = gpd.read_file(paths).set_crs(epsg=4326, inplace=True)
    lines = gpd.read_file(pathl).set_crs(epsg=4326, inplace=True)

if option == "clean":
    pathg = home + clean_path + "/all_generators.geojson"
    paths = home + clean_path + "/all_substations.geojson"
    pathl = home + clean_path + "/all_lines.geojson"

    generators = gpd.read_file(pathg).set_crs(epsg=4326, inplace=True)
    substations = gpd.read_file(paths).set_crs(epsg=4326, inplace=True)
    lines = gpd.read_file(pathl).set_crs(epsg=4326, inplace=True)

if option == "line2network":
    paths = (
        home + b_network_path  + "/all_buses_build_network.csv"
    )
    pathl = (
        home + b_network_path  + "/all_lines_build_network.csv"
    )
    patht = (
        home + b_network_path  + "/all_transformers_build_network.csv"
    )

    substations = gpd.read_file(
        paths, GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO"
    )
    substations["geometry"] = gpd.points_from_xy(
        pd.to_numeric(substations.lon, downcast="float"),
        pd.to_numeric(substations.lat, downcast="float"),
    )
    substations = substations.set_crs(epsg=4326, inplace=True)
    lines = gpd.read_file(
        pathl, GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO"
    ).set_crs(epsg=4326, inplace=True)

    transformers = gpd.read_file(
        patht, GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO"
    ).set_crs(epsg=4326, inplace=True)

# Static plot

In [21]:
ax = geoplot.polyplot(nigeriaplot, edgecolor="grey", alpha=0.5, figsize=(5, 5)) # Expand figsize for larger image

if option == "line2network":
    lines.plot(figsize=(5, 5), markersize=10, color="green", alpha=0.3, ax=ax)
    substations.plot(figsize=(5, 5), markersize=3, color="blue", alpha=0.3, ax=ax)

else:
    generators.plot(figsize=(5, 5), markersize=10, color="orange", alpha=0.3, ax=ax)
    lines.plot(figsize=(5, 5), markersize=10, color="green", alpha=0.3, ax=ax)
    substations.plot(figsize=(5, 5), markersize=3, color="blue", alpha=0.3, ax=ax)

# Example of how to constrain to one country:
# lines.loc[lines['country'] == 'nigeria'].plot(figsize=(20, 20),markersize=25, color='red',alpha = 0.3, ax=ax)

# #plt.savefig('africa_transmission_and substations_110.png')

NameError: name 'geoplot' is not defined

# Interactive plot

In [22]:
# Choose the country you want to focus
country = "BO"

In [23]:
if option == "line2network":
    lines_ng = lines[lines["country"] == country]
    substations_ng = substations[substations["country"] == country]

else:
    lines_ng = lines[lines["country"] == country]
    substations_ng = substations[substations["country"] == country]
    generators_ng = generators[generators["Country"] == country]

# Map

In [24]:
# Somehow works with "if" when using display()

if option == "line2network":
    line2network_BL = substations_ng.hvplot( # Buses and Lines
        geo=True,
        size=10,
        frame_height=750,
        alpha=0.4,
        tiles="CartoLight",
        hover_cols=["bus_id"],
        color="orange",
    ) * lines_ng.hvplot(geo=True, alpha=0.4, hover_cols=["line_id"]).opts(
        active_tools=["pan", "wheel_zoom"]
    )
    # display (line2network_BL)     # show plot in the notebook
    hvplot.save(line2network_BL, "documentation/line2network_BL.html") # save plot in the path here

else:
    raw_clean = substations_ng.hvplot(
        geo=True,
        size=10,
        frame_height=750,
        alpha=0.4,
        tiles="CartoLight",
        hover_cols=["bus_id"],
        color="orange",
    ) * lines_ng.hvplot(
        geo=True, alpha=0.4, hover_cols=["line_id"]
    ) * generators_ng.hvplot(
        geo=True, alpha=0.4, color="green"
    ).opts(
        active_tools=["pan", "wheel_zoom"]
    )
    # display (raw_clean)     # show plot in the notebook
    hvplot.save(raw_clean, "documentation/raw_clean.html") # save plot in the path here

DataError: Supplied data does not contain specified dimensions, the following dimensions were not found: ['Longitude', 'Latitude']

PandasInterface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html

In [8]:
# With transformers
line2network_BLT = substations_ng.hvplot( # Buses, Lines and Transformers
    geo=True,
    size=10,
    frame_height=750,
    alpha=0.4,
    tiles="CartoLight",
    hover_cols=["bus_id"],
    color="orange",
) * lines_ng.hvplot(geo=True, alpha=0.4, hover_cols=["line_id"]).opts(
    active_tools=["pan", "wheel_zoom"]
) * transformers.hvplot(
    geo=True, linesize=30, color="red", alpha=0.4, hover_cols=["line_id"]
).opts(
    active_tools=["pan", "wheel_zoom"]
)
# display (line2network_BLT)     # show plot in the notebook
hvplot.save(line2network_BLT, "documentation/line2network_BLT.html") # save plot in the path here